In [1]:
import pandas as pd
import numpy as np
import math
import os
from matplotlib import pyplot as plt
import sys
from scipy.interpolate import griddata
from tqdm import tqdm
# sys.path.append('/Users/jiarong/Google Drive/codes/jiarongw-postprocessing/functions')
sys.path.append('/home/jiarong/research/postprocessing/functions/')
from Amplitude import Amplitude
from fio import readin

In [2]:
# Helper function 1 that read in one set of parameter with the time given in postfix_index_set
def iter_onecase(postfix_index_set, tag_set, tag_common):
    '''
    postfix_index_set: list of numbers
        A list of corresponding indices used to distinguish the cases.
    tag_set: list of strings
        A list of tags that distinguish each case, can be different from the index set 
    
    '''

    # Instantiate amplitude as a list
    amplitude = []
    
    for i,(index,tag) in enumerate(zip(postfix_index_set, tag_set)):   
        # Use the enhanced read-in function   
        # Rewrite the line according to the file naming 
        # filename = 'filename_common' + '%g' %postfix_index + file_format
        filename = './eta' + '%g' %index
        # Rewrite this line according to the delimiter and the fields attributes
        eta, exists = readin(filename, table_delimiter = ',')
        if exists:
            eta.rename(columns={'pos':'eta'}, inplace=True)
            # Instantiate a amplitude class member
            ampl = Amplitude(eta[['x', 'eta', 'f']])
            amplitude.append(ampl)
    return amplitude, exists

In [3]:
# Helper function 2 that iterates through different parameter set 
def iter_para_newversion(para_pair_set, para_name, common_path, postfix_index, tag):
    amplitude_set = []
    energydirect_set = []
    for p in tqdm(para_pair_set):
        # Assemble the directory name corresponding to each parameter set
        # Example: dirname = './linear_m5B0Ustar0.5ak0.05Bo3.45Re31000.LEVEL10'
        dirname = 'linear_m5B0'
        for i,name in enumerate(para_name):
            if name == 'Re':
                dirname += name + '%g' %p[i] + '.'
            else:
                dirname += name + '%g' %p[i]  
        # Change path 
        os.chdir(common_path + dirname)
        # Call iter_onecase, pass directory name as a parameter
        amplitude, exists = iter_onecase(postfix_index_set=postfix_index, tag_set=tag, tag_common = 't = ')
        # If the file does not exceed, do not append
        if exists:
            amplitude_set.append(amplitude)
        energy, exists = readin('budgetWaterwind.dat', table_delimiter = ' ', skipn = 100)
        if exists:
            # HERE IS THE DIFFERENCE! 
            # total = energy.ke/2 + energy.gpe
            total = energy.ke + energy.gpe
            diss = np.zeros(energy.shape[0])
            for i, row in energy.iterrows():
                if i == 0:
                    diss[i] = 0
                    last_t = row['t'] # record t in last row
                else:
                    diss[i] = diss[i-1] + row['dissipation'] * (row['t'] - last_t)
                    last_t = row['t']
            energy.insert(len(energy.columns), "total", total, True)
            energy.insert(len(energy.columns), "total_diss", total+diss, True)
            energydirect_set.append(energy)       
    return amplitude_set, energydirect_set